In [1]:
#!/usr/bin/env python3

In [2]:
import socket
import os
import json
import  time
import tensorflow as tf
import numpy as np
from   nltk.translate.bleu_score import sentence_bleu
import random
import  re
import sys
import argparse

from   Config import Config
import Util
import Logger
from   Communication import Client
from ImagesInfo import ImagesInfo
from TimeKeeper import TimeKeeper

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('-s', '--server', action='store', type=str, required=False)
parser.add_argument('-t', '--test_number', action='store', type=int, required=False)
parser.add_argument('-v', '--verbose', action='store', type=int, required=False)
args, unknown = parser.parse_known_args()
print(args.server)

server_ip = args.server
test_number = args.test_number
verbose = args.verbose

if(verbose == None):
    verbose = 1

if(test_number == None):
    test_number = 2

test_scenarios = {1:"Complete jpg file buffer transfer", 2:"Decoded image buffer transfer"}

print("Test scenario = %d %s" % (test_number, test_scenarios[test_number]))


None
Test scenario = 2 Decoded image buffer transfer


In [4]:
Logger.set_log_level(verbose)
tk = TimeKeeper()
cfg = Config(server_ip)
client = Client(cfg)
imagesInfo = ImagesInfo(cfg)


In WSL


In [5]:
#!/usr/bin/env python3
import os
import numpy as np
import tensorflow as tf
from   tensorflow.keras import layers,Model
import pickle5 as pickle
from   tensorflow.keras.preprocessing.text import Tokenizer
from   tensorflow.keras.activations import tanh
from   tensorflow.keras.activations import softmax
from   numpy import float32
import json
import time

from Config import Config
import Logger
from Communication import Server
import Util
class Encoder(Model):
    def __init__(self,embed_dim):
        super(Encoder, self).__init__()
        # build your Dense layer with relu activation
        self.dense = tf.keras.layers.Dense(embed_dim, activation='relu')
        
    def call(self, features):
        # extract the features from the image shape: (batch, 8*8, embed_dim)
        features = self.dense(features)
        return features    

class Attention_model(Model):
    def __init__(self, units):
        super(Attention_model, self).__init__()
        self.units=units
        # build your Dense layer
        self.W1 = tf.keras.layers.Dense(units)
        # build your Dense layer
        self.W2 = tf.keras.layers.Dense(units)
        # build your final Dense layer with unit 1
        # self.V = tf.keras.layers.Dense(1, activation='softmax')
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        # features shape: (batch_size, 8*8, embedding_dim)
        # hidden shape: (batch_size, hidden_size)
        
        # Expand the hidden shape to shape: (batch_size, 1, hidden_size)
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        # build your score funciton to shape: (batch_size, 8*8, units)
        score = tanh(self.W1(features) + self.W2(hidden_with_time_axis))
        # extract your attention weights with shape: (batch_size, 8*8, 1)
        score = self.V(score)
        attention_weights = softmax(score, axis=1)

        # shape: create the context vector with shape (batch_size, 8*8,embedding_dim)
        context_vector = attention_weights * features
        # reduce the shape to (batch_size, embedding_dim)
        # context_vector = tf.reduce_sum(context_vector, axis=1)
        context_vector = tf.reduce_mean(context_vector, axis=1)

        return context_vector, attention_weights   
class Decoder(Model):
    def __init__(self, embed_dim, units, vocab_size, cfg):
        super(Decoder, self).__init__()
        self.cfg = cfg
        self.units = units
        # iniitalise your Attention model with units
        self.attention = Attention_model(self.units)
        # build your Embedding layer
        self.embed = tf.keras.layers.Embedding(vocab_size, self.cfg.embedding_dim)
        self.gru = tf.keras.layers.GRU(self.units,return_sequences=True,return_state=True,recurrent_initializer='glorot_uniform')
        # build your Dense layer
        self.d1 = tf.keras.layers.Dense(self.units)
        # build your Dense layer
        self.d2 = tf.keras.layers.Dense(vocab_size)


    def call(self,x,features, hidden):
        #create your context vector & attention weights from attention model
        context_vector, attention_weights = self.attention(features, hidden)
        # embed your input to shape: (batch_size, 1, embedding_dim)
        embed = self.embed(x)
        # Concatenate your input with the context vector from attention layer. 
        # Shape: (batch_size, 1, embedding_dim + embedding_dim)
        embed = tf.concat([tf.expand_dims(context_vector, 1), embed], axis=-1)
        # Extract the output & hidden state from GRU layer. 
        # Output shape : (batch_size, max_length, hidden_size)
        output, state = self.gru(embed)
        output = self.d1(output)
        # shape : (batch_size * max_length, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2])) 
        # shape : (batch_size * max_length, vocab_size)
        output = self.d2(output)

        return output, state, attention_weights

    def init_state(self, batch_size):
        return tf.zeros((batch_size, self.units))
class TailModel:
    def __init__(self,cfg):
        self.cfg = cfg

        with open(self.cfg.saved_model_path + '/tokenizer.pickle', 'rb') as handle:
            self.tokenizer = pickle.load(handle)
        
        with open(self.cfg.saved_model_path + '/image_features_extract_model.json', 'r') as json_file:
            json_savedModel= json_file.read()
        self.image_features_extract_model = tf.keras.models.model_from_json(json_savedModel)
        self.image_features_extract_model.load_weights(self.cfg.saved_model_path + '/image_features_extract_model.h5')
        vocab_size = cfg.max_tokenized_words + 1

        s = tf.zeros([32, 64, 2048], tf.int32)

        self.encoder=Encoder(self.cfg.embedding_dim)

        self.decoder=Decoder(self.cfg.embedding_dim, self.cfg.units, vocab_size,cfg)

        features = self.encoder(s)

        hidden = self.decoder.init_state(batch_size=self.cfg.batch_size)
        dec_input = tf.expand_dims([self.tokenizer.word_index['<start>']] * self.cfg.batch_size, 1)

        predictions, hidden_out, attention_weights= self.decoder(dec_input, features, hidden)

        self.decoder.load_weights(self.cfg.saved_model_path + "/decoder.h5")
        self.encoder.load_weights(self.cfg.saved_model_path + "/encoder.h5")

    def evaluate(self,image):
        attention_features_shape = 64
        attention_plot = np.zeros((self.cfg.MAX_SEQ_LENGTH, attention_features_shape))

        hidden = self.decoder.init_state(batch_size=1)

        # process the input image to desired format before extracting features
        temp_input = tf.expand_dims(image, 0) 
        # Extract features using our feature extraction model
        img_tensor_val = self.extract_image_features(temp_input)
        # img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

        # extract the features by passing the input to encoder
        features = self.encoder(img_tensor_val)

        dec_input = tf.expand_dims([self.tokenizer.word_index['<start>']], 0)
        result = []

        for i in range(self.cfg.MAX_SEQ_LENGTH):
            # get the output from decoder
            predictions, hidden, attention_weights = self.decoder(dec_input, features, hidden)

            attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

            #extract the predicted id(embedded value) which carries the max value
            predicted_id = tf.argmax(tf.transpose(predictions))
            predicted_id = predicted_id.numpy()[0]
            # map the id to the word from tokenizer and append the value to the result list
            result.append(self.tokenizer.index_word[predicted_id])

            if self.tokenizer.index_word[predicted_id] == '<end>':
                return result, attention_plot,predictions

            dec_input = tf.expand_dims([predicted_id], 0)

        attention_plot = attention_plot[:len(result), :]
        return result, attention_plot,predictions

    def process_image_file(self,msg,shape):
        temp_file = '/tmp/temp.bin'
        f = open(temp_file, "wb")
        f.write(msg)
        f.close()

        image_tensor,label = Util.read_image(temp_file,[])

        t0 = time.perf_counter()
        result, attention_plot,pred_test  = self.evaluate(image_tensor)
        t1 = time.perf_counter() - t0
        pred_caption=' '.join(result).rsplit(' ', 1)[0]

        send_json_dict = {}
        send_json_dict['response'] = 'OK'
        send_json_dict['pred_caption'] = pred_caption
        send_json_dict['tail_model_time'] = t1

        app_json = json.dumps(send_json_dict)

        return str(app_json)

    def process_image_tensor(self,msg,shape):
        generated_np_array = np.frombuffer(msg, dtype=float32)
        generated_image_np_array = generated_np_array.reshape(shape)
        image_tensor = tf.convert_to_tensor(generated_image_np_array, dtype=tf.float32)

        t0 = time.perf_counter()
        result, attention_plot,pred_test  = self.evaluate(image_tensor)
        t1 = time.perf_counter() - t0
        pred_caption=' '.join(result).rsplit(' ', 1)[0]

        send_json_dict = {}
        send_json_dict['response'] = 'OK'
        send_json_dict['pred_caption'] = pred_caption
        send_json_dict['tail_model_time'] = t1

        app_json = json.dumps(send_json_dict)

        return str(app_json)
        
    def extract_image_features(self, sample_img_batch):
        features = self.image_features_extract_model(sample_img_batch)
        features = tf.reshape(features, [sample_img_batch.shape[0],8*8, 2048])
        return features

# cfg = Config(None)
tailModel = TailModel(cfg)
# server = Server(cfg, tailModel)
# server.register_callback('data',tailModel.process_image_tensor)
# server.register_callback('file',tailModel.process_image_file)
# server.accept_connections()

# server.callbacks


In [6]:
if False:
    with open(file_name, 'rb') as file_t:
        blob_data = bytearray(file_t.read())
        send_json_dict = {}
        send_json_dict['data_type'] = 'file'
        send_json_dict['file_name'] = file_name
        send_json_dict['data_size'] = (len(blob_data))
        # send_json_dict['data_buffer'] = blob_data
        app_json = json.dumps(send_json_dict)
        print(str(app_json))

        t0= time.perf_counter()
        client.send_data(str(app_json), blob_data)
        t1 = time.perf_counter() - t0

        print("Time to send file: %.3f" % (t1))
def evaluate_file_over_server(file_name):
    with open(file_name, 'rb') as file_t:
        byte_buffer_to_send = bytearray(file_t.read())
        send_json_dict = {}
        send_json_dict['data_type'] = 'file'
        send_json_dict['file_name'] = file_name
        send_json_dict['data_size'] = (len(byte_buffer_to_send))
        send_json_dict['data_shape'] = "(%d,)" % (len(byte_buffer_to_send))
        # send_json_dict['data_buffer'] = blob_data

        app_json = json.dumps(send_json_dict)

        tk.logInfo(img_path, tk.I_BUFFER_SIZE, len(byte_buffer_to_send))

        tk.logTime(img_path, tk.E_START_COMMUNICATION)

        # response = client.send_data(str(app_json), byte_buffer_to_send)
        response = tailModel.process_image_file(byte_buffer_to_send, "(%d,)" % (len(byte_buffer_to_send)))

        tk.logTime(img_path, tk.E_STOP_COMMUNICATION)

        response = json.loads(response)

        pred_caption = response['pred_caption']
        tail_model_time = response['tail_model_time']
        tk.logInfo(img_path, tk.I_TAIL_MODEL_TIME, tail_model_time)

        return pred_caption, [], []

In [7]:
# tf.compat.v1.disable_eager_execution()

In [8]:
def evaluate_over_server(file_name):
    image_tensor,caption = Util.read_image(file_name,'')

    image_np_array = image_tensor.numpy()

    byte_buffer_to_send = image_np_array.tobytes()
    type(byte_buffer_to_send)

    send_json_dict = {}
    send_json_dict['data_type'] = 'data'
    send_json_dict['file_name'] = file_name
    send_json_dict['data_size'] = (len(byte_buffer_to_send))
    send_json_dict['data_shape'] = image_np_array.shape

    app_json = json.dumps(send_json_dict)

    tk.logInfo(img_path, tk.I_BUFFER_SIZE, len(byte_buffer_to_send))

    tk.logTime(img_path, tk.E_START_COMMUNICATION)

    # response = client.send_data(str(app_json), byte_buffer_to_send)
    response = tailModel.process_image_tensor(byte_buffer_to_send,image_np_array.shape)

    tk.logTime(img_path, tk.E_STOP_COMMUNICATION)

    response = json.loads(response)

    pred_caption = response['pred_caption']
    tail_model_time = response['tail_model_time']
    tk.logInfo(img_path, tk.I_TAIL_MODEL_TIME, tail_model_time)

    return pred_caption, [], []

In [9]:

total_time = 0.0
max_test_images = cfg.total_test_images
for i in range(max_test_images):
    Logger.event_print("")
    random_num = random.randint(0,max_test_images-1)
    img_path = imagesInfo.getImagePath(random_num)
    # image = io.imread(img_path)
    # plt.imshow(image)

    real_caption = imagesInfo.getCaption(random_num)

    tk.startRecord(img_path)
    tk.logTime(img_path, tk.E_START_CLIENT_PROCESSING)

    if(test_number == 1):
        pred_caption, attention_plot,pred_test = evaluate_file_over_server(img_path)
    if(test_number == 2):
        pred_caption, attention_plot,pred_test = evaluate_over_server(img_path)

    tk.logTime(img_path, tk.E_STOP_CLIENT_PROCESSING)

    real_caption=Util.filt_text(real_caption)      

    reference = imagesInfo.getAllCaptions(img_path)
    candidate = pred_caption.split()

    score = sentence_bleu(reference, candidate, weights=[1]) #set your weights)

    tk.logInfo(img_path, tk.I_BLEU, score)
    tk.logInfo(img_path, tk.I_REAL_CAPTION, real_caption)
    tk.logInfo(img_path, tk.I_PRED_CAPTION, pred_caption)
    tk.finishRecord(img_path)

    Logger.event_print("BLEU: %.2f" % (score))
    Logger.event_print ('Real: %s' % (real_caption))
    Logger.event_print ('Pred: %s' % (pred_caption))

tk.printAll()
tk.summary()




BLEU: 0.50
Real: two women walking through a parking lot carrying luggage
Pred: a woman is walking down the street

BLEU: 0.82
Real: a large tractor truck with a painting on the back of the cab
Pred: a truck with a skateboard on the back of a truck

BLEU: 0.35
Real: an nypd police car sitting on broadway near a stoplight
Pred: a busy city street with a street with a street light on a street

BLEU: 0.60
Real: a baby puts a tooth brush in its mouth
Pred: a young child brushing her teeth

BLEU: 0.25
Real: wires traffic and street signs at the corner of 51st and 72nd
Pred: a street sign on a pole on a pole on a pole on a pole on a pole on a pole on a pole

BLEU: 0.75
Real: a large jetliner parked on an airport tarmac
Pred: a large white airplane parked on the tarmac

BLEU: 0.70
Real: person holding a cut red grapefruit showing a large seed
Pred: a close up of an orange on a wooden table

BLEU: 0.70
Real: the fresh orange has been cut and is ready to be eaten
Pred: a close up of an orange 